In [1]:
from common import *

# importing dependencies and load data.
data = load_data_from_cvs("estadao_noticias_eleicao.csv")

SyntaxError: invalid syntax (common.py, line 221)

In [ ]:
data = data.fillna('')

In [ ]:
# joyning column of title and sub-title of artile with column of content.
data['articles'] = data['titulo']  + ' ' + data['subTitulo'] + ' ' + data['conteudo']

In [ ]:
# normalizing and tokenizing articles.
data['articles'] = data['articles'].apply(normalize)
data['tokens'] = data['articles'].apply(tokenize)

In [ ]:
# applying summarization to articles to produce inverted index.
inverted_index = summarize(data['tokens'], data['idNoticia'])

In [ ]:
search_with_vectorial_model('segundo turno', inverted_index)

In [ ]:
search_with_tf('segundo turno', inverted_index)

In [ ]:
search_with_tf_idf('segundo turno', inverted_index)

In [ ]:
search_with_bm25('segundo turno', inverted_index)

In [5]:
import numpy as np

def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

In [28]:
import pandas
import ast
gabarito = pandas.read_csv('gabarito.csv')

to_array = lambda text: ast.literal_eval(text)

# parsing string to array
gabarito.busca_binaria = gabarito.busca_binaria.apply(to_array)
gabarito.tf = gabarito.tf.apply(to_array)
gabarito.tfidf = gabarito.tfidf.apply(to_array)
gabarito.bm25 = gabarito.bm25.apply(to_array)


# create a dataframe with results
suaresposta = {'str_busca': 'segundo turno', 'busca_binaria': [[1, 2, 3, 7, 8]], 'tf': [[2744, 7, 2112, 2388, 1052]],
              'tfidf': [[2744, 7, 2112, 2388, 1052]], 'bm25': [[2744,2112, 7672, 2388, 2178]]}
suaresposta = pandas.DataFrame(data=suaresposta)


print("Precisão para consulta 'segundo turno' usando busca binaria %.2f%%" % (100 * mapk(gabarito.busca_binaria, suaresposta.busca_binaria, k=5)))

print("Precisão para consulta 'segundo turno' usando busca binaria %.2f%%" % (100 * mapk(gabarito.tf, suaresposta.tf, k=5)))

print("Precisão para consulta 'segundo turno' usando busca binaria %.2f%%" % (100 * mapk(gabarito.tfidf, suaresposta.tfidf, k=5)))

print("Precisão para consulta 'segundo turno' usando busca binaria %.2f%%" % (100 * mapk(gabarito.bm25, suaresposta.bm25, k=5)))

Precisão para consulta 'segundo turno' usando busca binaria 20.00%
Precisão para consulta 'segundo turno' usando busca binaria 80.00%
Precisão para consulta 'segundo turno' usando busca binaria 48.33%
Precisão para consulta 'segundo turno' usando busca binaria 100.00%
